# Classifiers

## Preparation

In [27]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense, LSTM, SimpleRNN, GRU, Bidirectional
from tensorflow.keras.models import save_model 

nltk.data.path.append("D:\\Environment\\nltk_data")

### Data reading/cleaning

In [2]:
df_train = pd.read_csv('../data/WELFake_clean_train.csv')
df_test = pd.read_csv('../data/WELFake_clean_test.csv')
df_train.head(5)

,Unnamed: 0,title,text,label
0,45905,Exclusive: Foreign Isis Fighters Defend Mosul ...,\nForeign fighters for Isis are choosing to ...,1
1,37291,JUDGE JEANINE UNLOADS On Hillary: “How Did You...,You don t want to miss a second of Judge Jeani...,1
2,46730,Gunman attacks Saudi security forces at gate o...,RIYADH (Reuters) - Two Saudi guards were shot ...,0
3,66327,Indian Software Mogul: Hire Americans Now Beca...,A leading Indian software entrepreneur says In...,0
4,58329,Rep. Diaz-Balart: Liberals Against Trump Who F...,Florida Congressman Mario attacked the “doub...,0


In [3]:
df_test.head(5)

,Unnamed: 0,title,text,label
0,50990,BLM Rapper & Bill’s Alleged Son Have Nasty Sur...,BLM Rapper & Bill’s Alleged Son Have Nasty Sur...,1
1,41146,US Airstrike Killed Five Al-Qaeda Members in Y...,Get short URL 0 2 0 0 The US military killed f...,1
2,48389,"Zach Johnson, Pieters share lead at Firestone",(Reuters) - Late birdies from Thomas Pieters a...,0
3,55759,Re: WOW! What Josh Earnest admitted about Obam...,WOW! What Josh Earnest admitted about Obamacar...,1
4,21412,Memorial Day provides respite from VA controve...,Memorial Day is a time to remember those who g...,0


In [4]:
def clean_text(text):
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # Remove URLs
    text = re.sub(r'[^A-Za-z\s]', '', text)  # Remove special characters and numbers
    text = text.lower()  # Convert to lowercase
    return text

In [5]:
df_train['title'] = df_train['title'].apply(clean_text)
df_train['text'] = df_train['text'].apply(clean_text)
df_test['title'] = df_test['title'].apply(clean_text)
df_test['text'] = df_test['text'].apply(clean_text)

df_train['title'] = df_train['title'].apply(word_tokenize)
df_train['text'] = df_train['text'].apply(word_tokenize)
df_test['title'] = df_test['title'].apply(word_tokenize)
df_test['text'] = df_test['text'].apply(word_tokenize)

stop_words = set(stopwords.words('english'))
df_train['title'] = df_train['title'].apply(lambda x: [word for word in x if word not in stop_words])
df_train['text'] = df_train['text'].apply(lambda x: [word for word in x if word not in stop_words])
df_test['title'] = df_test['title'].apply(lambda x: [word for word in x if word not in stop_words])
df_test['text'] = df_test['text'].apply(lambda x: [word for word in x if word not in stop_words])

lemmatizer = WordNetLemmatizer()
df_train['title'] = df_train['title'].apply(lambda x: [lemmatizer.lemmatize(token) for token in x])
df_train['text'] = df_train['text'].apply(lambda x: [lemmatizer.lemmatize(token) for token in x])
df_test['title'] = df_test['title'].apply(lambda x: [lemmatizer.lemmatize(token) for token in x])
df_test['text'] = df_test['text'].apply(lambda x: [lemmatizer.lemmatize(token) for token in x])

df_train['joined'] = df_train['title'].apply(lambda x: ' '.join(x)) + ' ' + df_train['text'].apply(lambda x: ' '.join(x))
df_test['joined'] = df_test['title'].apply(lambda x: ' '.join(x)) + ' ' + df_test['text'].apply(lambda x: ' '.join(x))

X_train = df_train['joined']
y_train = df_train['label']
X_test = df_test['joined']
y_test = df_test['label']

vectorizer = TfidfVectorizer(max_features=10000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

X_train

0        exclusive foreign isi fighter defend mosul fro...
1        judge jeanine unloads hillary go dead broke wo...
2        gunman attack saudi security force gate jeddah...
3        indian software mogul hire american trump oppo...
4        rep diazbalart liberal trump favored obamas cu...
                               ...                        
56629    applause boo kerry urge congress ratify pacifi...
56630    tingle leg nbc paid chris matthew staffer sexu...
56631    u government share technical detail north kore...
56632    trump history corruption mindboggling clinton ...
56633    facezam app let stranger stalk facebook profil...
Name: joined, Length: 56634, dtype: object

## Standard Machine Learning Methods

### Logistic Regression

In [6]:
def train_logistic_regression(X_train, y_train):
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)
    return model

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    print('Accuracy:', accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))

In [7]:
lr_model = train_logistic_regression(X_train_tfidf, y_train)
evaluate_model(lr_model, X_test_tfidf, y_test)

Accuracy: 0.9538809237940532
              precision    recall  f1-score   support

           0       0.96      0.95      0.95      6907
           1       0.95      0.96      0.96      7252

    accuracy                           0.95     14159
   macro avg       0.95      0.95      0.95     14159
weighted avg       0.95      0.95      0.95     14159



### Naive Bayes

In [8]:
def train_naive_bayes(X_train, y_train):
    model = MultinomialNB()
    model.fit(X_train, y_train)
    return model

In [9]:
nb_model = train_naive_bayes(X_train_tfidf, y_train)
evaluate_model(nb_model, X_test_tfidf, y_test)

Accuracy: 0.8529557172116675
              precision    recall  f1-score   support

           0       0.86      0.83      0.85      6907
           1       0.85      0.87      0.86      7252

    accuracy                           0.85     14159
   macro avg       0.85      0.85      0.85     14159
weighted avg       0.85      0.85      0.85     14159



### Support Vector Machines

In [14]:
def train_svm(X_train, y_train):
    model = SVC(kernel='linear', max_iter=1000)
    model.fit(X_train, y_train)
    return model

In [15]:
svm_model = train_svm(X_train_tfidf, y_train)
evaluate_model(svm_model, X_test_tfidf, y_test)

d:\Environment\anaconda\envs\deepmachinelearning\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Accuracy: 0.8973091320008475
              precision    recall  f1-score   support

           0       0.88      0.91      0.90      6907
           1       0.91      0.89      0.90      7252

    accuracy                           0.90     14159
   macro avg       0.90      0.90      0.90     14159
weighted avg       0.90      0.90      0.90     14159



### Random Forest

In [16]:
def train_random_forest(X_train, y_train):
    model = RandomForestClassifier(n_estimators=100, random_state=44)
    model.fit(X_train, y_train)
    return model

In [17]:
rf_model = train_random_forest(X_train_tfidf, y_train)
evaluate_model(rf_model, X_test_tfidf, y_test)

Accuracy: 0.954163429620736
              precision    recall  f1-score   support

           0       0.96      0.95      0.95      6907
           1       0.95      0.96      0.96      7252

    accuracy                           0.95     14159
   macro avg       0.95      0.95      0.95     14159
weighted avg       0.95      0.95      0.95     14159



## Deep Learning

### Data prep

In [18]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
maxlen = 500
X_train_pad = pad_sequences(X_train_seq, maxlen=maxlen)
X_test_pad = pad_sequences(X_test_seq, maxlen=maxlen)

### Convolutional NN

In [19]:
def train_cnn(X_train, y_train):
    model = Sequential()
    model.add(Embedding(input_dim=10000, output_dim=128, input_length=maxlen))
    model.add(Conv1D(128, 5, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)
    return model

def evaluate_dl_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_pred = (y_pred > 0.5).astype(int)
    print('Accuracy:', accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))

In [20]:
cnn_model = train_cnn(X_train_pad, y_train)
evaluate_dl_model(cnn_model, X_test_pad, y_test)

Epoch 1/10
708/708 [==============================] - 10s 6ms/step - loss: 0.1855 - accuracy: 0.9224 - val_loss: 0.0936 - val_accuracy: 0.9659
Epoch 2/10
708/708 [==============================] - 4s 6ms/step - loss: 0.0510 - accuracy: 0.9830 - val_loss: 0.0982 - val_accuracy: 0.9666
Epoch 3/10
708/708 [==============================] - 4s 6ms/step - loss: 0.0148 - accuracy: 0.9954 - val_loss: 0.1812 - val_accuracy: 0.9560
Epoch 4/10
708/708 [==============================] - 4s 6ms/step - loss: 0.0098 - accuracy: 0.9969 - val_loss: 0.1801 - val_accuracy: 0.9660
Epoch 5/10
708/708 [==============================] - 4s 6ms/step - loss: 0.0103 - accuracy: 0.9966 - val_loss: 0.1990 - val_accuracy: 0.9642
Epoch 6/10
708/708 [==============================] - 4s 6ms/step - loss: 0.0095 - accuracy: 0.9968 - val_loss: 0.1889 - val_accuracy: 0.9670
Epoch 7/10
708/708 [==============================] - 4s 6ms/step - loss: 0.0057 - accuracy: 0.9982 - val_loss: 0.1999 - val_accuracy: 0.9674
Epoch

### Recurrent NN

In [21]:
def train_rnn(X_train, y_train):
    model = Sequential()
    model.add(Embedding(input_dim=10000, output_dim=128, input_length=maxlen))
    model.add(SimpleRNN(128))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)
    return model

In [22]:
rnn_model = train_rnn(X_train_pad, y_train)
evaluate_dl_model(rnn_model, X_test_pad, y_test)

Epoch 1/10
708/708 [==============================] - 294s 415ms/step - loss: 0.3868 - accuracy: 0.8169 - val_loss: 0.3986 - val_accuracy: 0.8104
Epoch 2/10
708/708 [==============================] - 282s 398ms/step - loss: 0.2940 - accuracy: 0.8727 - val_loss: 0.2450 - val_accuracy: 0.9023
Epoch 3/10
708/708 [==============================] - 288s 407ms/step - loss: 0.2828 - accuracy: 0.8811 - val_loss: 0.3157 - val_accuracy: 0.8700
Epoch 4/10
708/708 [==============================] - 223s 314ms/step - loss: 0.2437 - accuracy: 0.9019 - val_loss: 0.3034 - val_accuracy: 0.8899
Epoch 5/10
708/708 [==============================] - 253s 358ms/step - loss: 0.1988 - accuracy: 0.9252 - val_loss: 0.2598 - val_accuracy: 0.9054
Epoch 6/10
708/708 [==============================] - 236s 334ms/step - loss: 0.1468 - accuracy: 0.9488 - val_loss: 0.2511 - val_accuracy: 0.9096
Epoch 7/10
708/708 [==============================] - 230s 326ms/step - loss: 0.1286 - accuracy: 0.9563 - val_loss: 0.2718 -

### LSTM NN

In [23]:
def train_lstm(X_train, y_train):
    model = Sequential()
    model.add(Embedding(input_dim=10000, output_dim=128, input_length=maxlen))
    model.add(LSTM(128))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)
    return model

In [24]:
lstm_model = train_lstm(X_train_pad, y_train)
evaluate_dl_model(lstm_model, X_test_pad, y_test)

Epoch 1/10
708/708 [==============================] - 27s 36ms/step - loss: 0.2619 - accuracy: 0.8959 - val_loss: 0.2490 - val_accuracy: 0.8947
Epoch 2/10
708/708 [==============================] - 24s 33ms/step - loss: 0.1383 - accuracy: 0.9491 - val_loss: 0.1511 - val_accuracy: 0.9433
Epoch 3/10
708/708 [==============================] - 24s 34ms/step - loss: 0.0914 - accuracy: 0.9676 - val_loss: 0.1567 - val_accuracy: 0.9470
Epoch 4/10
708/708 [==============================] - 24s 34ms/step - loss: 0.1330 - accuracy: 0.9479 - val_loss: 0.1098 - val_accuracy: 0.9620
Epoch 5/10
708/708 [==============================] - 24s 34ms/step - loss: 0.0373 - accuracy: 0.9868 - val_loss: 0.1444 - val_accuracy: 0.9564
Epoch 6/10
708/708 [==============================] - 24s 34ms/step - loss: 0.0160 - accuracy: 0.9953 - val_loss: 0.1238 - val_accuracy: 0.9663
Epoch 7/10
708/708 [==============================] - 24s 33ms/step - loss: 0.0082 - accuracy: 0.9976 - val_loss: 0.1530 - val_accuracy:

## Model Saving

In [28]:
joblib.dump(lr_model, '../classifiers/logisticregression.pkl')
joblib.dump(nb_model, '../classifiers/naivebayes.pkl')
joblib.dump(svm_model, '../classifiers/svm_model.pkl')
joblib.dump(rf_model, '../classifiers/rf_model.pkl')

['../classifiers/rf_model.pkl']

In [29]:
cnn_model.save('../classifiers/cnn.h5')
rnn_model.save('../classifiers/rnn.h5')
lstm_model.save('../classifiers/lstm_model.h5')